In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import tensorflow as tf
import psycopg2
import numpy as np

pd.set_option('display.max_columns',85)

# Import data from Database

In [2]:
conn = psycopg2.connect("dbname=ipo-database user=postgres password=0nlineSQL")

In [3]:
master_df = pd.read_sql_query("SELECT * from master_data;", conn)
master_df.head()

,symbol,trade_date,issuer,lead_jointlead_managers,offer_price,opening_price,firstday_close,firstday_percent_pxchng,dollar_change_opening,dollar_change_close,star_ratings,performed,asset_type,company_name,exchange,currency,country,sector,industry,address,three_mth_date,three_mth_ipo,price_change,three_mth_return,price_gain_loss,fiscaldate_end,reported_currency,gross_profit,total_revenue,costof_revenue,costof_goodservices_sold,operating_income,selling_gen_admin,research_dev,operating_expenses,investment_income_net,net_interest_income,interest_income,interest_expense,noninterest_income,othernon_operatingincome,depreciation,...,deferred_revenue,current_debt,shortterm_debt,total_noncurrent_liabilities,capital_lease_obligations,longterm_debt,current_longterm_debt,longterm_debt_noncurrent,short_longterm_debt_total,other_current_liabilities,other_noncurrent_liabilities,total_shareholder_equity,treasury_stock,retained_earnings,common_stock,common_stockshares_outstanding,operating_cashflow,payments_foroperating_activities,proceeds_fromoperating_activities,changein_operating_liabilities,changein_operating_assets,dpr_depletion_amort,capital_expenditures,changein_receivables,changein_inventory,profit_loss,cashflow_frominvestment,cashflow_fromfinancing,proceeds_fromrepayment_shortterm_debt,payments_forrepurchase_commonstock,payments_forrepurchase_equity,payments_forrepurchase_preferredstock,div_payout,div_payout_commonstock,div_payout_preferredstock,proceeds_fromissuance_commonstock,proceeds_fromissuance_longterm_debtcapital_secnet,proceeds_fromissuance_preferredstock,proceeds_fromrepurchase_equity,proceeds_fromsale_treasurystock,changein_cash_cashequivalents,changein_exchangerate
0,BSN.U,2020-09-11,Broadstone Acquisition,Citigroup,$10.00,$9.84,$9.60,-0.04,-$0.16,-$0.40,1.0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LEAP.U,2020-09-11,Ribbit LEAP,JPMorgan,$10.00,$11.20,$11.55,0.16,$1.20,$1.55,1.0,Missed,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SNPR.U,2020-09-11,Tortoise Acquisition Corp. II,Barclays/ Goldman Sachs,$10.00,$10.35,$10.51,0.05,$0.35,$0.51,1.0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TWCTU,2020-09-11,TWC Tech Holdings II,Citigroup/ Deutsche Bank Securities,$10.00,$10.08,$10.07,0.01,$0.08,$0.07,1.0,None,Common Stock,TWC Tech Holdings II Corp,NASDAQ,USD,USA,None,None,"Four Embarcadero Center, San Francisco, CA, Un...",2020-12-11,10.52,0.44,0.043651,Gain,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,242.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NSH.U,2020-09-10,NavSight Holdings,Credit Suisse,$10.00,$10.00,$10.00,0.00,$0.00,$0.00,1.0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
list(master_df)

['symbol',
 'trade_date',
 'issuer',
 'lead_jointlead_managers',
 'offer_price',
 'opening_price',
 'firstday_close',
 'firstday_percent_pxchng',
 'dollar_change_opening',
 'dollar_change_close',
 'star_ratings',
 'performed',
 'asset_type',
 'company_name',
 'exchange',
 'currency',
 'country',
 'sector',
 'industry',
 'address',
 'three_mth_date',
 'three_mth_ipo',
 'price_change',
 'three_mth_return',
 'price_gain_loss',
 'fiscaldate_end',
 'reported_currency',
 'gross_profit',
 'total_revenue',
 'costof_revenue',
 'costof_goodservices_sold',
 'operating_income',
 'selling_gen_admin',
 'research_dev',
 'operating_expenses',
 'investment_income_net',
 'net_interest_income',
 'interest_income',
 'interest_expense',
 'noninterest_income',
 'othernon_operatingincome',
 'depreciation',
 'dpr_and_amort',
 'income_beforetax',
 'incometax_expense',
 'interest_debt_expense',
 'netincome_cont_operations',
 'comprehensive_income_netoftax',
 'ebit',
 'ebitda',
 'netincome',
 'total_assets',
 't

## Calculated Columns

In [5]:
# Net profit margin
master_df["net_profit_margin"] = master_df["netincome"]/master_df["total_revenue"]
master_df["net_profit_margin"]

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
        ..
3465   NaN
3466   NaN
3467   NaN
3468   NaN
3469   NaN
Name: net_profit_margin, Length: 3470, dtype: float64

In [6]:
# Calculate Gross Profit Margin = gross_profit/total_revenue
master_df["gross_profit_margin"] = master_df["gross_profit"]/master_df["total_revenue"]
master_df["gross_profit_margin"]

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
        ..
3465   NaN
3466   NaN
3467   NaN
3468   NaN
3469   NaN
Name: gross_profit_margin, Length: 3470, dtype: float64

### Calculate Debt Asset Ratio:

The formula for the debt to asset ratio is as follows: Debt/Asset = (Short-term Debt + Long-term Debt) / Total Assets. Where: Total Assets may include all current and non-current assets on the company's balance sheet, or may only include certain assets such as Property, Plant & Equipment (PP&E)

In [7]:
# Debt to Asset Ratio
master_df["debt_asset_ratio"] = master_df["short_longterm_debt_total"]/master_df["total_assets"]
master_df

,symbol,trade_date,issuer,lead_jointlead_managers,offer_price,opening_price,firstday_close,firstday_percent_pxchng,dollar_change_opening,dollar_change_close,star_ratings,performed,asset_type,company_name,exchange,currency,country,sector,industry,address,three_mth_date,three_mth_ipo,price_change,three_mth_return,price_gain_loss,fiscaldate_end,reported_currency,gross_profit,total_revenue,costof_revenue,costof_goodservices_sold,operating_income,selling_gen_admin,research_dev,operating_expenses,investment_income_net,net_interest_income,interest_income,interest_expense,noninterest_income,othernon_operatingincome,depreciation,...,total_noncurrent_liabilities,capital_lease_obligations,longterm_debt,current_longterm_debt,longterm_debt_noncurrent,short_longterm_debt_total,other_current_liabilities,other_noncurrent_liabilities,total_shareholder_equity,treasury_stock,retained_earnings,common_stock,common_stockshares_outstanding,operating_cashflow,payments_foroperating_activities,proceeds_fromoperating_activities,changein_operating_liabilities,changein_operating_assets,dpr_depletion_amort,capital_expenditures,changein_receivables,changein_inventory,profit_loss,cashflow_frominvestment,cashflow_fromfinancing,proceeds_fromrepayment_shortterm_debt,payments_forrepurchase_commonstock,payments_forrepurchase_equity,payments_forrepurchase_preferredstock,div_payout,div_payout_commonstock,div_payout_preferredstock,proceeds_fromissuance_commonstock,proceeds_fromissuance_longterm_debtcapital_secnet,proceeds_fromissuance_preferredstock,proceeds_fromrepurchase_equity,proceeds_fromsale_treasurystock,changein_cash_cashequivalents,changein_exchangerate,net_profit_margin,gross_profit_margin,debt_asset_ratio
0,BSN.U,2020-09-11,Broadstone Acquisition,Citigroup,$10.00,$9.84,$9.60,-0.04,-$0.16,-$0.40,1.0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LEAP.U,2020-09-11,Ribbit LEAP,JPMorgan,$10.00,$11.20,$11.55,0.16,$1.20,$1.55,1.0,Missed,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SNPR.U,2020-09-11,Tortoise Acquisition Corp. II,Barclays/ Goldman Sachs,$10.00,$10.35,$10.51,0.05,$0.35,$0.51,1.0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TWCTU,2020-09-11,TWC Tech Holdings II,Citigroup/ Deutsche Bank Securities,$10.00,$10.08,$10.07,0.01,$0.08,$0.07,1.0,None,Common Stock,TWC Tech Holdings II Corp,NASDAQ,USD,USA,None,None,"Four Embarcadero Center, San Francisco, CA, Un...",2020-12-11,10.52,0.44,0.043651,Gain,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,242.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,NSH.U,2020-09-10,NavSight Holdings,Credit Suisse,$10.00,$10.00,$10.00,0.00,$0.00,$0.00,1.0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

### Calculate Current Ratio:

This is a financial KPI that measure the company's ability to pay off its short term financial obligations in one year.

The formula for Current Ratio is Total Current Assets / Total Current Liabilities.

In [8]:
# Current Ratio
master_df["current_ratio"] = master_df["total_current_assets"]/master_df["total_current_liabilities"]
master_df

,symbol,trade_date,issuer,lead_jointlead_managers,offer_price,opening_price,firstday_close,firstday_percent_pxchng,dollar_change_opening,dollar_change_close,star_ratings,performed,asset_type,company_name,exchange,currency,country,sector,industry,address,three_mth_date,three_mth_ipo,price_change,three_mth_return,price_gain_loss,fiscaldate_end,reported_currency,gross_profit,total_revenue,costof_revenue,costof_goodservices_sold,operating_income,selling_gen_admin,research_dev,operating_expenses,investment_income_net,net_interest_income,interest_income,interest_expense,noninterest_income,othernon_operatingincome,depreciation,...,capital_lease_obligations,longterm_debt,current_longterm_debt,longterm_debt_noncurrent,short_longterm_debt_total,other_current_liabilities,other_noncurrent_liabilities,total_shareholder_equity,treasury_stock,retained_earnings,common_stock,common_stockshares_outstanding,operating_cashflow,payments_foroperating_activities,proceeds_fromoperating_activities,changein_operating_liabilities,changein_operating_assets,dpr_depletion_amort,capital_expenditures,changein_receivables,changein_inventory,profit_loss,cashflow_frominvestment,cashflow_fromfinancing,proceeds_fromrepayment_shortterm_debt,payments_forrepurchase_commonstock,payments_forrepurchase_equity,payments_forrepurchase_preferredstock,div_payout,div_payout_commonstock,div_payout_preferredstock,proceeds_fromissuance_commonstock,proceeds_fromissuance_longterm_debtcapital_secnet,proceeds_fromissuance_preferredstock,proceeds_fromrepurchase_equity,proceeds_fromsale_treasurystock,changein_cash_cashequivalents,changein_exchangerate,net_profit_margin,gross_profit_margin,debt_asset_ratio,current_ratio
0,BSN.U,2020-09-11,Broadstone Acquisition,Citigroup,$10.00,$9.84,$9.60,-0.04,-$0.16,-$0.40,1.0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LEAP.U,2020-09-11,Ribbit LEAP,JPMorgan,$10.00,$11.20,$11.55,0.16,$1.20,$1.55,1.0,Missed,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SNPR.U,2020-09-11,Tortoise Acquisition Corp. II,Barclays/ Goldman Sachs,$10.00,$10.35,$10.51,0.05,$0.35,$0.51,1.0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TWCTU,2020-09-11,TWC Tech Holdings II,Citigroup/ Deutsche Bank Securities,$10.00,$10.08,$10.07,0.01,$0.08,$0.07,1.0,None,Common Stock,TWC Tech Holdings II Corp,NASDAQ,USD,USA,None,None,"Four Embarcadero Center, San Francisco, CA, Un...",2020-12-11,10.52,0.44,0.043651,Gain,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,242.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,4.138819
4,NSH.U,2020-09-10,NavSight Holdings,Credit Suisse,$10.00,$10.00,$10.00,0.00,$0.00,$0.00,1.0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

### Drop rows with N/As and infinite values

In [9]:
master_df.dropna(subset=["net_profit_margin"])

,symbol,trade_date,issuer,lead_jointlead_managers,offer_price,opening_price,firstday_close,firstday_percent_pxchng,dollar_change_opening,dollar_change_close,star_ratings,performed,asset_type,company_name,exchange,currency,country,sector,industry,address,three_mth_date,three_mth_ipo,price_change,three_mth_return,price_gain_loss,fiscaldate_end,reported_currency,gross_profit,total_revenue,costof_revenue,costof_goodservices_sold,operating_income,selling_gen_admin,research_dev,operating_expenses,investment_income_net,net_interest_income,interest_income,interest_expense,noninterest_income,othernon_operatingincome,depreciation,...,capital_lease_obligations,longterm_debt,current_longterm_debt,longterm_debt_noncurrent,short_longterm_debt_total,other_current_liabilities,other_noncurrent_liabilities,total_shareholder_equity,treasury_stock,retained_earnings,common_stock,common_stockshares_outstanding,operating_cashflow,payments_foroperating_activities,proceeds_fromoperating_activities,changein_operating_liabilities,changein_operating_assets,dpr_depletion_amort,capital_expenditures,changein_receivables,changein_inventory,profit_loss,cashflow_frominvestment,cashflow_fromfinancing,proceeds_fromrepayment_shortterm_debt,payments_forrepurchase_commonstock,payments_forrepurchase_equity,payments_forrepurchase_preferredstock,div_payout,div_payout_commonstock,div_payout_preferredstock,proceeds_fromissuance_commonstock,proceeds_fromissuance_longterm_debtcapital_secnet,proceeds_fromissuance_preferredstock,proceeds_fromrepurchase_equity,proceeds_fromsale_treasurystock,changein_cash_cashequivalents,changein_exchangerate,net_profit_margin,gross_profit_margin,debt_asset_ratio,current_ratio
6,SVACU,2020-09-10,Starboard Value Acquisition,UBS Investment Bank/ Stifel/ Cowen,$10.00,$10.00,$10.00,0.00,$0.00,$0.00,1.0,None,Common Stock,Starboard Value Acquisition Corp,NASDAQ,USD,USA,Financial Services,Shell Companies,"777 Third Avenue, New York, NY, United States,...",2020-12-10,10.45,0.45,0.045000,Gain,2020-09-30,USD,0.0,0.000000e+00,0.000000e+00,0.000000e+00,-2.007380e+05,5.280100e+04,0.0,2.007380e+05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.0,3.884905e+08,0.0,-1.770650e+05,0.0,4.042345e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-inf,NaN,0.000000,11.147009
11,INAQU,2020-09-02,INSU Acquisition Corp. II,Cantor,$10.00,$10.20,$10.26,0.03,$0.20,$0.26,1.0,None,Common Stock,INSU Acquisition Corp. II,NASDAQ,USD,USA,Financial Services,Shell Companies,"2929 Arch Street, Philadelphia, PA, United Sta...",2020-12-02,12.92,2.72,0.266667,Gain,2020-09-30,USD,0.0,0.000000e+00,0.000000e+00,0.000000e+00,-8.500100e+04,0.000000e+00,0.0,8.500100e+04,0.0,1386.0,1386.0,0.0,0.0,0.0,0.0,...,0.0,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.0,2.211063e+08,0.0,-8.473900e+04,0.0,3.138667e+07,-300113.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-inf,NaN,0.000000,35.208778
15,AUVI,2020-08-31,Applied UV,"Network 1 Financial Securities,",$5.00,$5.75,$11.60,1.32,$0.75,$6.60,0.0,None,Common Stock,"Applied UV, Inc",NASDAQ,USD,USA,Consumer Cyclical,"Furnishings, Fixtures & Appliances","150 N. Macquesten Parkway, Mount Vernon, NY, U...",2020-11-30,5.18,-0.57,-0.099130,Loss,2020-09-30,USD,78178.0,1.560633e+06,1.482455e+06,1.482455e+06,-8.949670e+05,9.251080e+05,48037.0,2.455600e+06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,532585.0,3.175300e+05,99297.0,0.0,416827.0,8.537970e+05,0.0,5.445513e+06,0.0,1.745910e+05,633.0,6.334308e+06,-429897.0,0.0,0.0,0.0,0.0,9182.0,55814.0,0.0,-45156.0,0.0,-5.581400e+04,4927529.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.573314,0.050094,0.053702,4.352835
16,HCDI,2020-08-28,Harbor Custom Development,ThinkEquity (a division of Fordham Financial M...,$6.00,$5.50,$7.50,0.25,-$0.50,$1.50,1.0,None,Common Stock,"Harbor Custom Development, Inc",NASDAQ,USD,USA,Real Estate,Real Estate-Development,"11505 Burnham D

In [10]:
master_df.dropna(subset=["debt_asset_ratio"])

,symbol,trade_date,issuer,lead_jointlead_managers,offer_price,opening_price,firstday_close,firstday_percent_pxchng,dollar_change_opening,dollar_change_close,star_ratings,performed,asset_type,company_name,exchange,currency,country,sector,industry,address,three_mth_date,three_mth_ipo,price_change,three_mth_return,price_gain_loss,fiscaldate_end,reported_currency,gross_profit,total_revenue,costof_revenue,costof_goodservices_sold,operating_income,selling_gen_admin,research_dev,operating_expenses,investment_income_net,net_interest_income,interest_income,interest_expense,noninterest_income,othernon_operatingincome,depreciation,...,capital_lease_obligations,longterm_debt,current_longterm_debt,longterm_debt_noncurrent,short_longterm_debt_total,other_current_liabilities,other_noncurrent_liabilities,total_shareholder_equity,treasury_stock,retained_earnings,common_stock,common_stockshares_outstanding,operating_cashflow,payments_foroperating_activities,proceeds_fromoperating_activities,changein_operating_liabilities,changein_operating_assets,dpr_depletion_amort,capital_expenditures,changein_receivables,changein_inventory,profit_loss,cashflow_frominvestment,cashflow_fromfinancing,proceeds_fromrepayment_shortterm_debt,payments_forrepurchase_commonstock,payments_forrepurchase_equity,payments_forrepurchase_preferredstock,div_payout,div_payout_commonstock,div_payout_preferredstock,proceeds_fromissuance_commonstock,proceeds_fromissuance_longterm_debtcapital_secnet,proceeds_fromissuance_preferredstock,proceeds_fromrepurchase_equity,proceeds_fromsale_treasurystock,changein_cash_cashequivalents,changein_exchangerate,net_profit_margin,gross_profit_margin,debt_asset_ratio,current_ratio
3,TWCTU,2020-09-11,TWC Tech Holdings II,Citigroup/ Deutsche Bank Securities,$10.00,$10.08,$10.07,0.01,$0.08,$0.07,1.0,None,Common Stock,TWC Tech Holdings II Corp,NASDAQ,USD,USA,None,None,"Four Embarcadero Center, San Francisco, CA, Un...",2020-12-11,10.52,0.44,0.043651,Gain,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,242.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,4.138819
6,SVACU,2020-09-10,Starboard Value Acquisition,UBS Investment Bank/ Stifel/ Cowen,$10.00,$10.00,$10.00,0.00,$0.00,$0.00,1.0,None,Common Stock,Starboard Value Acquisition Corp,NASDAQ,USD,USA,Financial Services,Shell Companies,"777 Third Avenue, New York, NY, United States,...",2020-12-10,10.45,0.45,0.045000,Gain,2020-09-30,USD,0.0,0.0,0.0,0.0,-200738.0,52801.0,0.0,200738.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,388490525.0,0.0,-177065.0,0.0,40423453.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-inf,NaN,0.000000,11.147009
11,INAQU,2020-09-02,INSU Acquisition Corp. II,Cantor,$10.00,$10.20,$10.26,0.03,$0.20,$0.26,1.0,None,Common Stock,INSU Acquisition Corp. II,NASDAQ,USD,USA,Financial Services,Shell Companies,"2929 Arch Street, Philadelphia, PA, United Sta...",2020-12-02,12.92,2.72,0.266667,Gain,2020-09-30,USD,0.0,0.0,0.0,0.0,-85001.0,0.0,0.0,85001.0,0.0,1386.0,1386.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,221106345.0,0.0,-84739.0,0.0,31386667.0,-300113.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-inf,NaN,0.000000,35.208778
15,AUVI,2020-08-31,Applied UV,"Network 1 Financial Securities,",$5.00,$5.75,$11.60,1.32,$0.75,$6.60,0.0,None,Common Stock,"Applied UV, Inc",NASDAQ,USD,USA,Consumer Cyclical,"Furnishings, Fixtures & Appliances","150 N. Macquesten Parkway, Mount Vernon, NY, U...",2020-11-30,5.18,-0.57,-0.099130,Loss,2020-09-30,USD,78178.0,1560633.0,1482455.0,1482455.0,-894967.0,925108.0,48037.0,2455600.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,532585.0,317530.0,99297.0,0.0,416827.0,853797.0,0.0,5445513.0,0.0,174591.0,633.0,6334308.0,-429897.0,0.0,0.0,0.0,0.0,9182.0,55814.0,0.0,-45156.0,0.0,-55814.0,4927529.0,0.0,0.

In [11]:
# master_df = master_df.dropna(subset=["gross_profit_margin","debt_asset_ratio","current_ratio"])
# master_df

In [12]:
# Remove infinity values from calculated columns dividing by 0
master_df = master_df.replace([-np.inf, np.inf], np.nan)
master_df

,symbol,trade_date,issuer,lead_jointlead_managers,offer_price,opening_price,firstday_close,firstday_percent_pxchng,dollar_change_opening,dollar_change_close,star_ratings,performed,asset_type,company_name,exchange,currency,country,sector,industry,address,three_mth_date,three_mth_ipo,price_change,three_mth_return,price_gain_loss,fiscaldate_end,reported_currency,gross_profit,total_revenue,costof_revenue,costof_goodservices_sold,operating_income,selling_gen_admin,research_dev,operating_expenses,investment_income_net,net_interest_income,interest_income,interest_expense,noninterest_income,othernon_operatingincome,depreciation,...,capital_lease_obligations,longterm_debt,current_longterm_debt,longterm_debt_noncurrent,short_longterm_debt_total,other_current_liabilities,other_noncurrent_liabilities,total_shareholder_equity,treasury_stock,retained_earnings,common_stock,common_stockshares_outstanding,operating_cashflow,payments_foroperating_activities,proceeds_fromoperating_activities,changein_operating_liabilities,changein_operating_assets,dpr_depletion_amort,capital_expenditures,changein_receivables,changein_inventory,profit_loss,cashflow_frominvestment,cashflow_fromfinancing,proceeds_fromrepayment_shortterm_debt,payments_forrepurchase_commonstock,payments_forrepurchase_equity,payments_forrepurchase_preferredstock,div_payout,div_payout_commonstock,div_payout_preferredstock,proceeds_fromissuance_commonstock,proceeds_fromissuance_longterm_debtcapital_secnet,proceeds_fromissuance_preferredstock,proceeds_fromrepurchase_equity,proceeds_fromsale_treasurystock,changein_cash_cashequivalents,changein_exchangerate,net_profit_margin,gross_profit_margin,debt_asset_ratio,current_ratio
0,BSN.U,2020-09-11,Broadstone Acquisition,Citigroup,$10.00,$9.84,$9.60,-0.04,-$0.16,-$0.40,1.0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LEAP.U,2020-09-11,Ribbit LEAP,JPMorgan,$10.00,$11.20,$11.55,0.16,$1.20,$1.55,1.0,Missed,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SNPR.U,2020-09-11,Tortoise Acquisition Corp. II,Barclays/ Goldman Sachs,$10.00,$10.35,$10.51,0.05,$0.35,$0.51,1.0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TWCTU,2020-09-11,TWC Tech Holdings II,Citigroup/ Deutsche Bank Securities,$10.00,$10.08,$10.07,0.01,$0.08,$0.07,1.0,None,Common Stock,TWC Tech Holdings II Corp,NASDAQ,USD,USA,None,None,"Four Embarcadero Center, San Francisco, CA, Un...",2020-12-11,10.52,0.44,0.043651,Gain,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,242.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,4.138819
4,NSH.U,2020-09-10,NavSight Holdings,Credit Suisse,$10.00,$10.00,$10.00,0.00,$0.00,$0.00,1.0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

# Preprocess

In [13]:
IPO_df = master_df[["sector","industry","price_gain_loss","net_profit_margin"]]
IPO_df

,sector,industry,price_gain_loss,net_profit_margin
0,None,None,None,NaN
1,None,None,None,NaN
2,None,None,None,NaN
3,None,None,Gain,NaN
4,None,None,None,NaN
...,...,...,...,...
3465,None,None,None,NaN
3466,Other,Other,None,NaN
3467,Other,Other,None,NaN
3468,Other,Other,None,NaN


In [14]:
IPO_df.shape

(3470, 4)

In [15]:
IPO_df = IPO_df.dropna()
IPO_df.shape

(383, 4)

In [16]:
IPO_df.nunique()

sector                12
industry              77
price_gain_loss        2
net_profit_margin    335
dtype: int64

# Encoding

In [17]:
# Binary encode
IPO_df_encoded = pd.get_dummies(IPO_df, columns=["sector","industry","price_gain_loss"])
IPO_df_encoded.head()

,net_profit_margin,sector_Basic Materials,sector_Communication Services,sector_Consumer Cyclical,sector_Consumer Defensive,sector_Energy,sector_Financial Services,sector_Healthcare,sector_Industrials,sector_Other,sector_Real Estate,sector_Technology,sector_Utilities,industry_Advertising Agencies,industry_Aerospace & Defense,industry_Airlines,industry_Airports & Air Services,industry_Apparel Manufacturing,industry_Asset Management,industry_Auto & Truck Dealerships,industry_Auto Manufacturers,industry_Banks-Regional,industry_Biotechnology,industry_Building Materials,industry_Building Products & Equipment,industry_Capital Markets,industry_Chemicals,industry_Communication Equipment,industry_Computer Hardware,industry_Credit Services,industry_Diagnostics & Research,industry_Discount Stores,industry_Drug Manufacturers-Specialty & Generic,industry_Education & Training Services,industry_Electrical Equipment & Parts,industry_Electronic Gaming & Multimedia,industry_Entertainment,"industry_Furnishings, Fixtures & Appliances",industry_Gambling,industry_Health Information Services,industry_Industrial Distribution,industry_Information Technology Services,...,industry_Lodging,industry_Medical Care Facilities,industry_Medical Devices,industry_Medical Instruments & Supplies,industry_Metal Fabrication,industry_Mortgage Finance,industry_Oil & Gas Drilling,industry_Oil & Gas E&P,industry_Oil & Gas Equipment & Services,industry_Oil & Gas Midstream,industry_Oil & Gas Refining & Marketing,industry_Other,industry_Packaged Foods,industry_Packaging & Containers,industry_Pharmaceutical Retailers,industry_Pollution & Treatment Controls,industry_REIT-Diversified,industry_REIT-Mortgage,industry_Real Estate Services,industry_Real Estate-Development,industry_Recreational Vehicles,industry_Rental & Leasing Services,industry_Residential Construction,industry_Resorts & Casinos,industry_Restaurants,industry_Semiconductors,industry_Shell Companies,industry_Software-Application,industry_Software-Infrastructure,industry_Specialty Chemicals,industry_Specialty Industrial Machinery,industry_Specialty Retail,industry_Staffing & Employment Services,industry_Telecom Services,industry_Travel Services,industry_Trucking,industry_Utilities-Regulated Gas,industry_Utilities-Regulated Water,industry_Utilities-Renewable,industry_Waste Management,price_gain_loss_Gain,price_gain_loss_Loss
15,-0.573314,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
19,-0.577232,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
39,0.138040,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
51,-0.124847,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
53,-0.233533,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0


# Target Feature

In [18]:
y = IPO_df_encoded.price_gain_loss_Gain
X = IPO_df_encoded.drop(columns=["price_gain_loss_Gain","price_gain_loss_Loss"])

In [19]:
y

15      0
19      1
39      1
51      1
53      1
       ..
2722    1
2823    1
2907    1
2932    0
3020    1
Name: price_gain_loss_Gain, Length: 383, dtype: uint8

In [20]:
X

,net_profit_margin,sector_Basic Materials,sector_Communication Services,sector_Consumer Cyclical,sector_Consumer Defensive,sector_Energy,sector_Financial Services,sector_Healthcare,sector_Industrials,sector_Other,sector_Real Estate,sector_Technology,sector_Utilities,industry_Advertising Agencies,industry_Aerospace & Defense,industry_Airlines,industry_Airports & Air Services,industry_Apparel Manufacturing,industry_Asset Management,industry_Auto & Truck Dealerships,industry_Auto Manufacturers,industry_Banks-Regional,industry_Biotechnology,industry_Building Materials,industry_Building Products & Equipment,industry_Capital Markets,industry_Chemicals,industry_Communication Equipment,industry_Computer Hardware,industry_Credit Services,industry_Diagnostics & Research,industry_Discount Stores,industry_Drug Manufacturers-Specialty & Generic,industry_Education & Training Services,industry_Electrical Equipment & Parts,industry_Electronic Gaming & Multimedia,industry_Entertainment,"industry_Furnishings, Fixtures & Appliances",industry_Gambling,industry_Health Information Services,industry_Industrial Distribution,industry_Information Technology Services,...,industry_Internet Retail,industry_Leisure,industry_Lodging,industry_Medical Care Facilities,industry_Medical Devices,industry_Medical Instruments & Supplies,industry_Metal Fabrication,industry_Mortgage Finance,industry_Oil & Gas Drilling,industry_Oil & Gas E&P,industry_Oil & Gas Equipment & Services,industry_Oil & Gas Midstream,industry_Oil & Gas Refining & Marketing,industry_Other,industry_Packaged Foods,industry_Packaging & Containers,industry_Pharmaceutical Retailers,industry_Pollution & Treatment Controls,industry_REIT-Diversified,industry_REIT-Mortgage,industry_Real Estate Services,industry_Real Estate-Development,industry_Recreational Vehicles,industry_Rental & Leasing Services,industry_Residential Construction,industry_Resorts & Casinos,industry_Restaurants,industry_Semiconductors,industry_Shell Companies,industry_Software-Application,industry_Software-Infrastructure,industry_Specialty Chemicals,industry_Specialty Industrial Machinery,industry_Specialty Retail,industry_Staffing & Employment Services,industry_Telecom Services,industry_Travel Services,industry_Trucking,industry_Utilities-Regulated Gas,industry_Utilities-Regulated Water,industry_Utilities-Renewable,industry_Waste Management
15,-0.573314,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
19,-0.577232,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
39,0.138040,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
51,-0.124847,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
53,-0.233533,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2722,0.000000,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2823,0.383139,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [21]:
X.describe()

,net_profit_margin,sector_Basic Materials,sector_Communication Services,sector_Consumer Cyclical,sector_Consumer Defensive,sector_Energy,sector_Financial Services,sector_Healthcare,sector_Industrials,sector_Other,sector_Real Estate,sector_Technology,sector_Utilities,industry_Advertising Agencies,industry_Aerospace & Defense,industry_Airlines,industry_Airports & Air Services,industry_Apparel Manufacturing,industry_Asset Management,industry_Auto & Truck Dealerships,industry_Auto Manufacturers,industry_Banks-Regional,industry_Biotechnology,industry_Building Materials,industry_Building Products & Equipment,industry_Capital Markets,industry_Chemicals,industry_Communication Equipment,industry_Computer Hardware,industry_Credit Services,industry_Diagnostics & Research,industry_Discount Stores,industry_Drug Manufacturers-Specialty & Generic,industry_Education & Training Services,industry_Electrical Equipment & Parts,industry_Electronic Gaming & Multimedia,industry_Entertainment,"industry_Furnishings, Fixtures & Appliances",industry_Gambling,industry_Health Information Services,industry_Industrial Distribution,industry_Information Technology Services,...,industry_Internet Retail,industry_Leisure,industry_Lodging,industry_Medical Care Facilities,industry_Medical Devices,industry_Medical Instruments & Supplies,industry_Metal Fabrication,industry_Mortgage Finance,industry_Oil & Gas Drilling,industry_Oil & Gas E&P,industry_Oil & Gas Equipment & Services,industry_Oil & Gas Midstream,industry_Oil & Gas Refining & Marketing,industry_Other,industry_Packaged Foods,industry_Packaging & Containers,industry_Pharmaceutical Retailers,industry_Pollution & Treatment Controls,industry_REIT-Diversified,industry_REIT-Mortgage,industry_Real Estate Services,industry_Real Estate-Development,industry_Recreational Vehicles,industry_Rental & Leasing Services,industry_Residential Construction,industry_Resorts & Casinos,industry_Restaurants,industry_Semiconductors,industry_Shell Companies,industry_Software-Application,industry_Software-Infrastructure,industry_Specialty Chemicals,industry_Specialty Industrial Machinery,industry_Specialty Retail,industry_Staffing & Employment Services,industry_Telecom Services,industry_Travel Services,industry_Trucking,industry_Utilities-Regulated Gas,industry_Utilities-Regulated Water,industry_Utilities-Renewable,industry_Waste Management
count,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,...,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000,383.000000
mean,-19.726016,0.013055,0.060052,0.073107,0.028721,0.023499,0.075718,0.185379,0.044386,0.355091,0.023499,0.109661,0.007833,0.007833,0.002611,0.002611,0.002611,0.005222,0.007833,0.002611,0.007833,0.005222,0.099217,0.005222,0.002611,0.010444,0.002611,0.002611,0.005222,0.028721,0.018277,0.002611,0.010444,0.018277,0.005222,0.002611,0.002611,0.005222,0.002611,0.015666,0.002611,0.005222,...,0.018277,0.005222,0.002611,0.002611,0.031332,0.002611,0.002611,0.002611,0.002611,0.005222,0.007833,0.005222,0.002611,0.355091,0.007833,0.005222,0.005222,0.002611,0.005222,0.002611,0.013055,0.002611,0.002611,0.002611,0.002611,0.002611,0.005222,0.010444,0.002611,0.049608,0.036554,0.005222,0.002611,0.0

# Split Training and Testing sets

In [22]:
# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Resample

In [36]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

In [37]:
X_resampled.shape

(290, 90)

In [38]:
y_resampled.shape

(290,)

In [39]:
X.shape

(383, 90)

In [35]:
y.shape

(383,)

# Logistic Regression
our basic logistic regression parameters, which include:

- The solver parameter is set to 'lbfgs', which is an algorithm for learning and optimization. The particular solver isn't very important in this example, but note that a number of optimizers exist.
- The max_iter parameter will be set to 200 iterations, which will give the model sufficient opportunity to converge on effective weights

In [40]:
from sklearn.linear_model import LogisticRegression

# Define the logistic regression model
log_classifier = LogisticRegression(solver="lbfgs",max_iter=200)

# Train the model
log_classifier.fit(X_train,y_train)

# Evaluate the model
y_pred = log_classifier.predict(X_test)
print(f" Logistic regression model accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Logistic regression model accuracy: 0.573


In [41]:
from sklearn.linear_model import LogisticRegression

# Define the logistic regression model
log_classifier = LogisticRegression(solver="lbfgs",max_iter=200)

# Train the model
log_classifier.fit(X_resampled,y_resampled)

# Evaluate the model
y_pred = log_classifier.predict(X_test)
print(f" Logistic regression model accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Logistic regression model accuracy: 0.573


# Random Forest

In [24]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
print(f" Random forest predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Random forest predictive accuracy: 0.500


# SVM
- issues running when operating cash flow was added

In [25]:
# from sklearn.svm import SVC

# # Create the SVM model
# svm = SVC(kernel='linear')

# # Train the model
# svm.fit(X_train, y_train)

# # Evaluate the model
# y_pred = svm.predict(X_test_scaled)
# print(f" SVM model accuracy: {accuracy_score(y_test,y_pred):.3f}")

# Deep Learning

In [26]:
len(X_train_scaled[0])

90

In [27]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  number_input_features*1
# hidden_nodes_layer2 = number_input_features

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
# nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [28]:
# Train the model 
fit_model = nn.fit(X_train_scaled, y_train, epochs=100) 
# Evaluate the model using the test data 
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
9/9 [==============================] - 0s 1ms/step - loss: 0.8108 - accuracy: 0.5178
Epoch 2/100
9/9 [==============================] - 0s 2ms/step - loss: 0.6959 - accuracy: 0.5634
Epoch 3/100
9/9 [==============================] - 0s 2ms/step - loss: 0.6514 - accuracy: 0.6061
Epoch 4/100
9/9 [==============================] - 0s 1ms/step - loss: 0.6503 - accuracy: 0.6119
Epoch 5/100
9/9 [==============================] - 0s 2ms/step - loss: 0.6151 - accuracy: 0.6248
Epoch 6/100
9/9 [==============================] - 0s 1ms/step - loss: 0.6072 - accuracy: 0.6337
Epoch 7/100
9/9 [==============================] - 0s 1ms/step - loss: 0.5982 - accuracy: 0.6420
Epoch 8/100
9/9 [==============================] - 0s 2ms/step - loss: 0.5952 - accuracy: 0.6360
Epoch 9/100
9/9 [==============================] - 0s 1ms/step - loss: 0.5915 - accuracy: 0.6479
Epoch 10/100
9/9 [==============================] - 0s 1ms/step - loss: 0.5722 - accuracy: 0.6505
Epoch 11/100
9/9 [===========

9/9 [==============================] - 0s 997us/step - loss: 0.5215 - accuracy: 0.6664
Epoch 84/100
9/9 [==============================] - 0s 997us/step - loss: 0.5325 - accuracy: 0.6908
Epoch 85/100
9/9 [==============================] - 0s 1ms/step - loss: 0.5338 - accuracy: 0.6544
Epoch 86/100
9/9 [==============================] - 0s 1ms/step - loss: 0.5571 - accuracy: 0.6475
Epoch 87/100
9/9 [==============================] - 0s 997us/step - loss: 0.5336 - accuracy: 0.6397
Epoch 88/100
9/9 [==============================] - 0s 997us/step - loss: 0.5204 - accuracy: 0.6779
Epoch 89/100
9/9 [==============================] - 0s 996us/step - loss: 0.5361 - accuracy: 0.6901
Epoch 90/100
9/9 [==============================] - 0s 748us/step - loss: 0.5198 - accuracy: 0.7000
Epoch 91/100
9/9 [==============================] - 0s 997us/step - loss: 0.5159 - accuracy: 0.6971
Epoch 92/100
9/9 [==============================] - 0s 999us/step - loss: 0.5214 - accuracy: 0.6723
Epoch 93/100
9/9 